In [0]:
!pip install PyDrive

In [0]:
from gensim.models import KeyedVectors
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import json
import re

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1_cdWzw7BKlacbzTszPkI7IRsx6ChMp4V'})
download.GetContentFile('GoogleNews_Negative300.bin')
download = drive.CreateFile({'id': '1M8xBDXjqW4odEAnM-UurZpo6gv0olIsA'})
download.GetContentFile('queries_stopped.json')
download = drive.CreateFile({'id': '1fSJ_iYUzyFobgeM7-_iBNhcP1kjzzYip'})
download.GetContentFile('freebase-vectors-skipgram1000.bin')

In [0]:
word_vectors = KeyedVectors.load_word2vec_format("GoogleNews_Negative300.bin", binary=True)
#word_vectors = KeyedVectors.load_word2vec_format("freebase-vectors-skipgram1000.bin", binary=True)
#result = word_vectors.similar_by_word("New_York")
#print(result)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
queries = []
with open('queries_stopped.json') as json_file:
    data = json.load(json_file)
    queries.extend(list(data.values()))
freebase = False

In [0]:
def check_similarities(word, freebase):
  if freebase:
    word = "/en/" + word
  try:
    similarities = word_vectors.similar_by_word(word)
    # Add similar words above threshold to the query
    if freebase:
      similar_words = [x[4:] for (x,y) in similarities if y>=threshold]
    else:
      similar_words = [x for (x,y) in similarities if y>=threshold]
    return similar_words
  except KeyError:
    print("{0} is not in vocabulary, skipping word".format(word))
    return []

expanded_queries = []
threshold = 0.8
for query in queries:
  # Split into words
  new_query = []
  query_words = query.split()
  for i in range(len(query_words)):
    new_words = [query_words[i]]
    # Find similar words based on word2_vec
    #if len(word)>3:
    new_words.extend(check_similarities(query_words[i], freebase))
    # Repeat for bigrams:
    if i+1<len(query_words):
      new_words.extend(check_similarities("_".join(query_words[i:i+2]), freebase))
    new_query.extend(new_words)
  expanded_queries.append(" ".join(new_query))
print(expanded_queries)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Szechwan_dish is not in vocabulary, skipping word
dish_food is not in vocabulary, skipping word
food_cuisine is not in vocabulary, skipping word
roman_architecture is not in vocabulary, skipping word
finland_car is not in vocabulary, skipping word
car_industry is not in vocabulary, skipping word
industry_manufacturer is not in vocabulary, skipping word
manufacturer_saab is not in vocabulary, skipping word
saab_sisu is not in vocabulary, skipping word
france_second is not in vocabulary, skipping word
second_world is not in vocabulary, skipping word
world_war is not in vocabulary, skipping word
war_normandy is not in vocabulary, skipping word
normandy is not in vocabulary, skipping word
social_network is not in vocabulary, skipping word
network_group is not in vocabulary, skipping word
group_selection is not in vocabulary, skipping word
D_Day is not in vocabulary, skipping word
Day_normandy is not in vocabulary, skipping word
normandy is not in vocabulary, skipping word
normandy_invasion

In [0]:
i = 0
with open('queries_stopped.json') as json_file:
    data = json.load(json_file)
    for key, query in data.items():
      data[key] = expanded_queries[i]
      i += 1
    print(data)
    with open('queries_expanded_bigram_80.json', 'w+') as output:
      json.dump(data, output)

{'INEX_LD-2009022': 'Szechwan dish food cuisine cuisines', 'INEX_LD-2009039': 'roman architecture', 'INEX_LD-2009053': 'finland car industry manufacturer saab sisu', 'INEX_LD-2009061': 'france second third fourth fifth world war normandy', 'INEX_LD-2009062': 'social network networks group selection', 'INEX_LD-2009063': 'D Day normandy invasion', 'INEX_LD-2009074': 'web ranking scoring algorithm algorithms', 'INEX_LD-2009096': 'Eiffel', 'INEX_LD-2009111': 'europe solar photovoltaic solar_PV photovoltaic_solar power facility', 'INEX_LD-2009115': 'virtual museums', 'INEX_LD-2010004': 'Indian food', 'INEX_LD-2010014': 'composer composers museum Museum', 'INEX_LD-2010019': 'gallo roman architecture in paris', 'INEX_LD-2010020': 'electricity electricty source in France', 'INEX_LD-2010037': 'social network networks API', 'INEX_LD-2010043': 'List of films movies film from the surrealist category', 'INEX_LD-2010057': 'Einstein Relativity theory', 'INEX_LD-2010069': 'summer flowers', 'INEX_LD-20